In [155]:
import pandas as pd
import StkDataKeeper as sk
from pyecharts import Line, Kline, Grid, Overlap, Page, Bar
from pyecharts.base import Base
import pyecharts as pet

In [2]:
dk = sk.StkDataKeeper()
filename = '/root/notebooks/5min_603988.csv'
df = dk.read_csv_tu_5min(filename)
df = dk.read_csv(filename, index="date")

days = 2
data_length = 12*4*days
df = df.iloc[-(data_length):]
df.head().iloc[0].name

'2017-09-28 09:35'

In [3]:
v_lable = []

v_kline = []
v_line = []
v_volume = []

for i in range(data_length):
    v_kline.append([df.iloc[i].open, df.iloc[i].close, df.iloc[i].low, df.iloc[i].high])
    v_lable.append(df.iloc[i].name.split())
    v_line.append(df.iloc[i].close)
    v_volume.append(df.iloc[i].volume)

In [4]:
kline = Kline("K 线图示例")
kline.add("日K", v_lable, v_kline, is_datazoom_show=True)

In [12]:
line = Line("折线图示例")
line.add("", v_lable, v_line, mark_point=["average"], yaxis_min=min(v_line), is_datazoom_show=True)
#line.add("商家B", attr, v2, is_smooth=True, mark_line=["max", "average"])

In [13]:
bar = Bar("BAR")
bar.add("", v_lable, v_volume, is_datazoom_show=True, yaxis_max=max(v_volume)*3)
bar

In [14]:
# grid = Grid()
# grid.add(line, grid_bottom="60%", grid_left="60%")
# grid.add(kline, grid_top="60%", grid_left="60%")
# grid
# page = Page()
# page.add(line)
# page.add(kline)
# page

overlap = Overlap()
#overlap.add(kline)
overlap.add(line)
overlap.add(bar, is_add_yaxis=True, yaxis_index=1)
overlap

In [19]:
filename = "day_300191.csv"
dk = sk.StkDataKeeper()
df = dk.read_csv_tu_daily(filename)
df = df.iloc[-48:]

v_lable = []
v_volume = []
v_kdata = []
[(v_lable.append(index), 
  v_volume.append(df.loc[index].volume),
  v_kdata.append([df.loc[index].open, df.loc[index].close, df.loc[index].low, df.loc[index].high])
 ) for index in df.index]
i = 0

In [20]:
kline = Kline("K 线图示例")
kline.add("日K", v_lable, v_kdata, is_datazoom_show=True)

In [21]:
kline

In [22]:
import tushare as ts


In [23]:
def obtain_daily_data(sk_code, tail=0):
    try:
        df = ts.get_k_data(sk_code)
        df.set_index('date', inplace=True)
        if tail != 0:
            df = df[-(tail):]
        return df
    except:
        return NULL


In [28]:
df = obtain_daily_data("300191", 48)
df.tail()

,open,close,high,low,volume,code
date,,,,,,
2017-10-20,25.59,25.90,26.05,25.57,15383.0,300191
2017-10-23,25.95,25.75,26.03,25.31,18093.0,300191
2017-10-24,25.57,25.54,25.74,25.20,12389.0,300191
2017-10-25,25.55,25.84,25.98,25.45,14156.0,300191
2017-10-26,25.79,25.87,25.97,25.62,17175.0,300191


In [103]:
def draw_kline(df, title_top="0%", title_pos="0%"):
    v_lable = []
    v_volume = []
    v_kdata = []
    [(v_lable.append(index), 
      v_volume.append(df.loc[index].volume),
      v_kdata.append([df.loc[index].open, df.loc[index].close, df.loc[index].low, df.loc[index].high])
     ) for index in df.index]
    kline = Kline(df.iloc[0].code, title_top=title_top, title_pos=title_pos)#, width=400, height=300)
    kline.add("日K", v_lable, v_kdata, is_datazoom_show=True)
    return kline

In [104]:
kline = draw_kline(df, "10%", "10%")
kline

In [108]:
stk_list = ['000418', '002194', '002846', '300176', '300425', '300707', '603040', '603335', '002264', '002902', '603110']

In [130]:
column = 3
row = len(stk_list)//column + (1 if len(stk_list)%column != 0 else 0)

# every cell is grid_top, grid_bottom, grid_left, grid_right
positioning = []
pos_unit = 100/column
height_unit = 100/row
for i in range(row):
    for j in range(column):
        positioning.append(["{0:.0f}%".format(height_unit*i), 
                            "{0:.0f}%".format(100-height_unit*(i+1)), 
                            "{0:.0f}%".format(pos_unit*j),
                            "{0:.0f}%".format(100-pos_unit*(j+1))])
positioning

[['0%', '75%', '0%', '67%'],
 ['0%', '75%', '33%', '33%'],
 ['0%', '75%', '67%', '0%'],
 ['25%', '50%', '0%', '67%'],
 ['25%', '50%', '33%', '33%'],
 ['25%', '50%', '67%', '0%'],
 ['50%', '25%', '0%', '67%'],
 ['50%', '25%', '33%', '33%'],
 ['50%', '25%', '67%', '0%'],
 ['75%', '0%', '0%', '67%'],
 ['75%', '0%', '33%', '33%'],
 ['75%', '0%', '67%', '0%']]

In [131]:
df_list = []
for code in stk_list:
    df = obtain_daily_data(code, 15)
#     print (df.head())
    df_list.append(df)

grid_width, grid_height, grid_top, grid_bottom, grid_left, grid_right, title_top, title_pos

In [132]:
klines = []
enumerate(df_list)
for i, df in enumerate(df_list):

    kline = draw_kline(df, positioning[i][0], positioning[i][2])
    klines.append(kline)


In [133]:
klines[6]

In [157]:
grid = pet.Grid(width=1200)
# super(Grid, grid).width
# grid.height
for i, kline in enumerate(klines):
    #  print(positioning[i][0], positioning[i][1])
    grid.add(kline, 
             grid_top=positioning[i][0], 
             grid_bottom=positioning[i][1], 
             grid_left=positioning[i][2],
             grid_right=positioning[i][3]
            )
grid
# page = Page()
# for kline in klines:
#     page.add(kline)

# page

TypeError: __init__() got an unexpected keyword argument 'width'

In [86]:
positioning

[['0%', '0%'],
 ['0%', '33%'],
 ['0%', '67%'],
 ['33%', '0%'],
 ['33%', '33%'],
 ['33%', '67%'],
 ['67%', '0%'],
 ['67%', '33%'],
 ['67%', '67%']]